In [25]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

In [26]:
vector_dimension = 100

In [27]:
vector_a = np.random.randint(vector_dimension,size=vector_dimension) 
vector_b = np.random.randint(vector_dimension,size=vector_dimension)

In [28]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [29]:
mf = cl.mem_flags 
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_a) 
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_b) 

In [30]:
program = cl.Program(context, """ 
__kernel void vectorSum(__global const int *a_g, __global const int *b_g, __global int *res_g) { 
  int gid = get_global_id(0); 
  res_g[gid] = a_g[gid] + b_g[gid]; 
} 
""").build()

In [31]:
res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)

In [32]:
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

In [33]:
res_np = np.empty_like(vector_a)

In [34]:
cl.enqueue_copy(queue, res_np, res_g)

In [35]:
print ("PyOPENCL SUM OF TWO VECTORS")
print ("Platform Selected = %s" %platform.name )
print ("Device Selected = %s" %device.name)
print ("VECTOR LENGTH = %s" %vector_dimension)
print ("INPUT VECTOR A")
print (vector_a)
print ("INPUT VECTOR B")
print (vector_b)
print ("OUTPUT VECTOR RESULT A + B ")
print (res_np)

PyOPENCL SUM OF TWO VECTORS
Platform Selected = Apple
Device Selected = Intel(R) Core(TM) i5-7267U CPU @ 3.10GHz
VECTOR LENGTH = 100
INPUT VECTOR A
[98 28 17 14 26 36 99 21 38 40 34 48 16 16 43 93 46 91 44 13 36 97 46 18
 31 37 72 51 50 40 50 31 76 82 28 53 23  6 50 48 55 52 60 16 53 14 62 20
 90 18 17 30 18 38  8 41  0 50 54 12 93 19 39 64 90 48 17 83 67 34 71 76
 41 93 37 68 13 14 82 49 98 27  2 94 92 32 55 79 85 47 18 10  1 30 10  8
 64 10 11 64]
INPUT VECTOR B
[91  3 89 40 45 47 29 12 20 10  9  2  8 16 88 51 81 66 34 85 13 41 44 88
  0 96 23  2 13 21 13 51 47  8 86 79 64  5  9 20  2 78 45 60 81 19 91 72
  0 16 82 15 15 61 59 69 87 70 97 92 12 19 46  0 74 82 91 88 13 46 35 39
  1 63 18 55 94 78 23 87 44 20 10 70  2 46 44 21 82 91 98 24 42 52 37 14
 17 80 64 89]
OUTPUT VECTOR RESULT A + B 
[189  31 106  54  71  83 128  33  58  50  43  50  24  32 131 144 127 157
  78  98  49 138  90 106  31 133  95  53  63  61  63  82 123  90 114 132
  87  11  59  68  57 130 105  76 134  33 153  92  9

In [36]:
assert(la.norm(res_np - (vector_a + vector_b))) < 1e-5

AssertionError: 

In [37]:
print(vector_a + vector_b)


[189  31 106  54  71  83 128  33  58  50  43  50  24  32 131 144 127 157
  78  98  49 138  90 106  31 133  95  53  63  61  63  82 123  90 114 132
  87  11  59  68  57 130 105  76 134  33 153  92  90  34  99  45  33  99
  67 110  87 120 151 104 105  38  85  64 164 130 108 171  80  80 106 115
  42 156  55 123 107  92 105 136 142  47  12 164  94  78  99 100 167 138
 116  34  43  82  47  22  81  90  75 153]
